In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
# Limpa ambiente
!pip uninstall -y datasets peft transformers trl accelerate bitsandbytes

# Unsloth (fonte oficial)
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# Stack COMPATÍVEL com Unsloth Zoo (2025)
!pip install \
  peft>=0.13.0 \
  datasets==4.3.0 \
  transformers==4.57.3 \
  trl==0.18.2 \
  accelerate \
  bitsandbytes


Found existing installation: datasets 4.3.0
Uninstalling datasets-4.3.0:
  Successfully uninstalled datasets-4.3.0
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: trl 0.9.6
Uninstalling trl-0.9.6:
  Successfully uninstalled trl-0.9.6
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: bitsandbytes 0.49.0
Uninstalling bitsandbytes-0.49.0:
  Successfully uninstalled bitsandbytes-0.49.0
  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-0a8hb6c0/unsloth_e1190ebf266e46a78b5b8b0c37516f1b
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-0a8hb6c0/unsloth_e1190ebf266e46a78b5b8b0c37516f1b
  Resolved 

In [ ]:
import os
os.kill(os.getpid(), 9)


In [1]:
import trl, peft, datasets, transformers

print("trl:", trl.__version__)            # 0.18.2
print("peft:", peft.__version__)          # >= 0.13.x
print("datasets:", datasets.__version__)  # 4.3.0
print("transformers:", transformers.__version__)  # 4.57.3


trl: 0.18.2
peft: 0.18.0
datasets: 4.3.0
transformers: 4.57.3


In [2]:
import unsloth
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
import json
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments

DATA_PATH = "/content/drive/MyDrive/TC3 IA/nomalized_total.json"
OUTPUT_PATH_DATASET = "/content/drive/MyDrive/TC3 IA/output_QA.json"
max_seq_length = 1024
dtype = None
load_in_4bit = True
fourbit_models = [
    "unsloth/mistral-7b-v0.3-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/llama-3-8b-bnb-4bit",
    "unsloth/llama-3-8b-Instruct-bnb-4bit",
    "unsloth/llama-3-70b-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
]

/tmp/ipython-input-1430943130.py:1: UserWarning: WARNING: Unsloth should be imported before [trl, transformers, peft] to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Unsloth: Could not import trl.trainer.alignprop_trainer: Failed to import trl.trainer.alignprop_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base because of the following error (look up to see its traceback):
Failed to import diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion because of the following error (look up to see its traceback):
Failed to import diffusers.loaders.ip_adapter because of the following error (look up to see its traceback):
/usr/local/lib/python3.12/dist-packages/xformers/flash_attn_3/_C.so: undefined symbol: _ZNK3c106SymInt22maybe_as_int_slow_pathEv
Unsloth: Could not import trl.trainer.ddpo_trainer: Failed to import trl.trainer.ddpo_trainer because of the following error (look up to see its traceback):
Failed to import trl.models.modeling_sd_base beca

In [3]:
def format_dataset_into_model_input(data):
    # Inicializando as listas para armazenar os dados
    instructions = []
    inputs = []
    outputs = []

    # Processando o dataset de QA
    for item in data["medical_qa"]:
        question = item.get("question", "").strip()
        answer = item.get("normalized_answer", "").strip()

        if not question or not answer:
            continue

        instructions.append(
            "Answer the medical question based on scientific evidence."
        )
        inputs.append(question)
        outputs.append(answer)

    # Criando o dicionário final
    formatted_data = {
        "instruction": instructions,
        "input": inputs,
        "output": outputs
    }

    # Salvando o resultado em um arquivo JSON
    with open(OUTPUT_PATH_DATASET, "w", encoding="utf-8") as output_file:
        json.dump(formatted_data, output_file, indent=4, ensure_ascii=False)

    print(f"Dataset salvo em {OUTPUT_PATH_DATASET}")
    print(f"Total de exemplos: {len(outputs)}")


In [17]:
import json

with open(DATA_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

format_dataset_into_model_input(data)


model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
    device_map= None,
)


Dataset salvo em /content/drive/MyDrive/TC3 IA/output_QA.json
Total de exemplos: 6199
==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [18]:
  model = FastLanguageModel.get_peft_model(
      model,
      r = 16,
      target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                        "gate_proj", "up_proj", "down_proj"],
      lora_alpha = 16,
      lora_dropout = 0,
      bias = "none",

      use_gradient_checkpointing = "unsloth",
      random_state = 3407,
      use_rslora = False,
      loftq_config = None,
  )

In [19]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):

        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset

OUTPUT_PATH_DATASET = "/content/drive/MyDrive/TC3 IA/output_QA.json"
dataset = load_dataset("json", data_files=OUTPUT_PATH_DATASET, split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/6199 [00:00<?, ? examples/s]

In [23]:
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = True,
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        no_cuda=False,
        report_to = "none",
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [24]:

trainer_stats = trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 6,199 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Step,Training Loss
1,2.775900
2,2.715000
3,2.819000
4,2.508600
5,2.268200
6,1.871600
7,1.674200
8,1.548800
9,1.259300
10,1.245600


Unsloth: Will smartly offload gradients to save VRAM!


In [29]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
            "Answer the medical question based on scientific evidence.",
            "Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?",
            "",

    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nAnswer the medical question based on scientific evidence.\n\n### Input:\nDo mitochondria play a role in remodelling lace plant leaves during programmed cell death?\n\n### Response:\nMitochondria are involved in programmed cell death of lace plant leaves, and mitochondrial DNA damage may contribute to the process.']

In [30]:

FastLanguageModel.for_inference(model)
inputs = tokenizer(
[
    alpaca_prompt.format(
            "Answer the medical question based on scientific evidence.",
            "Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?",
            "",
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the medical question based on scientific evidence.

### Input:
Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?

### Response:
Lace plant leaves undergo programmed cell death (PCD) during development, and mitochondria play a role in the process.<|end_of_text|>


In [31]:
model.save_pretrained("/content/drive/MyDrive/TC3 IA/lora model") # Local saving
tokenizer.save_pretrained("/content/drive/MyDrive/TC3 IA/lora model")


('/content/drive/MyDrive/TC3 IA/lora model/tokenizer_config.json',
 '/content/drive/MyDrive/TC3 IA/lora model/special_tokens_map.json',
 '/content/drive/MyDrive/TC3 IA/lora model/tokenizer.json')

In [34]:
import json

qa_test = []

with open('/content/drive/MyDrive/TC3 IA/output_QA.json', 'r') as f:
    data = json.loads(f.read())

for inst, inp, out in zip(
    data["instruction"],
    data["input"],
    data["output"]
):
    qa_test.append({
        "instruction": inst,
        "input": inp,
        "output": out
    })


In [35]:
from unsloth import FastLanguageModel

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="/content/drive/MyDrive/TC3 IA/lora model",  # 🔴 SEU LoRA treinado
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

FastLanguageModel.for_inference(model)



==((====))==  Unsloth 2025.12.9: Fast Llama patching. Transformers: 4.57.3.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096, padding_idx=128255)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lor

In [36]:
inputs = tokenizer(
    [
        alpaca_prompt.format(
            "Answer the medical question based on scientific evidence.",
            "Does long-term use of proton pump inhibitors increase the risk of chronic kidney disease?",
            "",
        )
    ],
    return_tensors="pt",
).to("cuda")


In [37]:
from transformers import TextStreamer

text_streamer = TextStreamer(tokenizer)

_ = model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=128,
)


<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Answer the medical question based on scientific evidence.

### Input:
Does long-term use of proton pump inhibitors increase the risk of chronic kidney disease?

### Response:
Long-term use of proton pump inhibitors may be associated with increased risk of chronic kidney disease.<|end_of_text|>
